# Particle Categorical

In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

import tensorflow.keras as keras
import tensorflow.keras.backend as K
# from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization
# from tensorflow.keras.layers import Conv1D
# from tensorflow.keras.layers import Flatten, Reshape, Lambda
# from tensorflow.keras.utils import plot_model
# from tensorflow.keras import Model

import os
import os.path as osp
import sys

import numpy as np
#from scipy import linalg as LA

import matplotlib
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from utils.tf_sinkhorn import ground_distance_tf_nograd, sinkhorn_knopp_tf_scaling_stabilized_class
import utils.VAE_model_tools
from utils.VAE_model_tools import build_and_compile_annealing_vae, betaVAEModel, reset_metrics

import pandas
import matplotlib.pyplot as plt

import h5py
import pickle


1 Physical GPUs, 1 Logical GPUs


In [158]:
def create_dir(dir_path):
    ''' Creates a directory (or nested directories) if they don't exist.
    '''
    if not osp.exists(dir_path):
        os.makedirs(dir_path)

    return dir_path

output_dir = './data/'

def ptetaphiE_to_Epxpypz(jets):
    pt = jets[:,:,0]
    eta = jets[:,:,1]
    phi = jets[:,:,2]
    E = jets[:,:,3]
    
    px = pt * np.cos(phi)
    py = pt * np.sin(phi)
    pz = pt * np.sinh(eta)
    
    newjets = np.zeros(jets.shape)
    newjets[:,:,0] = E
    newjets[:,:,1] = px
    newjets[:,:,2] = py
    newjets[:,:,3] = pz
    
    return newjets

def ptetaphiE_to_ptyphim(jets):
    pt = jets[:,:,0]
    eta = jets[:,:,1]
    phi = jets[:,:,2]
    E = jets[:,:,3]
    
    pz = pt * np.sinh(eta)
    y = 0.5*np.nan_to_num(np.log((E+pz)/(E-pz)))
    
    msqr = np.square(E)-np.square(pt)-np.square(pz)
    msqr[np.abs(msqr) < 1e-6] = 0
    m = np.sqrt(msqr)
    
    newjets = np.zeros(jets.shape)
    newjets[:,:,0] = pt
    newjets[:,:,1] = y
    newjets[:,:,2] = phi
    newjets[:,:,3] = m
    
    return newjets
    
def ptyphim_to_ptetaphiE(jets):
    
    pt = jets[:,:,0]
    y = jets[:,:,1]
    phi = jets[:,:,2]
    m = jets[:,:,3]
    
    eta = np.nan_to_num(np.arcsinh(np.sinh(y)*np.sqrt(1+np.square(m/pt))))
    pz = pt * np.sinh(eta)
    E = np.sqrt(np.square(pz)+np.square(pt)+np.square(m))
    
    newjets = np.zeros(jets.shape)
    newjets[:,:,0] = pt
    newjets[:,:,1] = eta
    newjets[:,:,2] = phi
    newjets[:,:,3] = E
    
    return newjets
    
def center_jets_ptetaphiE(jets):
    cartesian_jets = ptetaphiE_to_Epxpypz(jets)
    sumjet_cartesian = np.sum(cartesian_jets,axis=1)
    
    sumjet_phi = np.arctan2(sumjet_cartesian[:,2],sumjet_cartesian[:,1])
    sumjet_y = 0.5*np.log((sumjet_cartesian[:,0] + sumjet_cartesian[:,-1])/(sumjet_cartesian[:,0] - sumjet_cartesian[:,-1]))
    
    ptyphim_jets = ptetaphiE_to_ptyphim(jets)
    #print(ptyphim_jets[:3,:,:])
    
    transformed_jets = np.copy(ptyphim_jets)
    transformed_jets[:,:,1] = ptyphim_jets[:,:,1] - sumjet_y[:,None]
    transformed_jets[:,:,2] = ptyphim_jets[:,:,2] - sumjet_phi[:,None]
    transformed_jets[:,:,2] = transformed_jets[:,:,2] + np.pi
    transformed_jets[:,:,2] = np.mod(transformed_jets[:,:,2],2*np.pi)
    transformed_jets[:,:,2] = transformed_jets[:,:,2] - np.pi

    transformed_jets[transformed_jets[:,:,0] == 0] = 0
    
    newjets = ptyphim_to_ptetaphiE(transformed_jets)
    return newjets
    

In [154]:
sumjet_phi

array([-1.95878748, -1.90034937])

In [155]:
jets = data[0:2]
this = center_jets_ptetaphiE(jets)

<ipython-input-153-5150e05f3be4>:36: RuntimeWarning: invalid value encountered in true_divide
  y = 0.5*np.nan_to_num(np.log((E+pz)/(E-pz)))
<ipython-input-153-5150e05f3be4>:57: RuntimeWarning: invalid value encountered in true_divide
  eta = np.nan_to_num(np.arcsinh(np.sinh(y)*np.sqrt(1+np.square(m/pt))))


In [156]:
this[0,:,2]

array([ 0.05336568, -0.19611154,  0.24988242,  0.14157883, -0.10185905,
        0.08940144, -0.28236875, -0.02938134, -0.07818067, -0.15213836,
        0.09155677,  0.5290762 ,  0.00659413, -0.29460946,  0.29623268,
        0.49414509, -0.73762267, -0.42383062, -0.91727924, -0.20876193,
        0.18704053,  0.21334067,  0.86678429, -0.57836097,  0.35393138,
       -0.63817282,  0.71705465,  0.73441442,  0.19266944, -0.23034891,
       -0.67779492,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [150]:
cartesian_jets = ptetaphiE_to_Epxpypz(jets)
sumjet_cartesian = np.sum(cartesian_jets,axis=1)

sumjet_phi = np.arctan2(sumjet_cartesian[:,2],sumjet_cartesian[:,1])
sumjet_y = 0.5*np.log((sumjet_cartesian[:,0] + sumjet_cartesian[:,-1])/(sumjet_cartesian[:,0] - sumjet_cartesian[:,-1]))
ptyphim_jets = ptetaphiE_to_ptyphim(jets)

transformed_jets = np.copy(ptyphim_jets)
transformed_jets[:,:,1] = ptyphim_jets[:,:,1] - sumjet_y[:,None]
transformed_jets[:,:,2] = ptyphim_jets[:,:,2] - sumjet_phi[:,None]
transformed_jets[:,:,2] = transformed_jets[:,:,2] + np.pi
transformed_jets[:,:,2] = np.mod(transformed_jets[:,:,2],2*np.pi)
transformed_jets[:,:,2] = transformed_jets[:,:,2] - np.pi

<ipython-input-142-452218278313>:36: RuntimeWarning: invalid value encountered in true_divide
  y = 0.5*np.nan_to_num(np.log((E+pz)/(E-pz)))


In [151]:
ptyphim_jets[0,:,2]

array([-1.9054218 , -2.15489902, -1.70890506, -1.81720865, -2.06064653,
       -1.86938604, -2.24115623, -1.98816882, -2.03696815, -2.11092584,
       -1.86723071, -1.42971128, -1.95219335, -2.25339694, -1.6625548 ,
       -1.46464238, -2.69641014, -2.3826181 , -2.87606672, -2.16754941,
       -1.77174695, -1.7454468 , -1.09200319, -2.53714845, -1.6048561 ,
       -2.5969603 , -1.24173283, -1.22437306, -1.76611804, -2.18913639,
       -2.6365824 ,  1.18280518,  1.18280518,  1.18280518,  1.18280518,
        1.18280518,  1.18280518,  1.18280518,  1.18280518,  1.18280518,
        1.18280518,  1.18280518,  1.18280518,  1.18280518,  1.18280518,
        1.18280518,  1.18280518,  1.18280518,  1.18280518,  1.18280518])

In [152]:
sumjet_phi[:,None]

array([[-1.95878748],
       [-1.90034937]])

In [133]:
ptyphim_jets[0]

array([[ 7.03576131e-01,  1.38958095e+00, -1.90542180e+00,
         1.20309630e-02],
       [ 2.16898474e-01,  1.50957744e+00, -2.15489902e+00,
         7.70915789e-03],
       [ 1.89505373e-02,  1.60033585e+00, -1.70890506e+00,
         1.53677432e-03],
       [ 1.53381211e-02,  1.67085073e+00, -1.81720865e+00,
         1.28216450e-03],
       [ 1.32579025e-02,  1.53381304e+00, -2.06064653e+00,
         7.10652757e-04],
       [ 9.37008207e-03,  1.48369369e+00, -1.86938604e+00,
         9.56582848e-04],
       [ 3.59286550e-03,  5.32778300e-01, -2.24115623e+00,
         4.98355408e-04],
       [ 3.44217527e-03,  8.95583761e-01, -1.98816882e+00,
         2.68313116e-04],
       [ 3.15475526e-03,  1.44198869e+00, -2.03696815e+00,
         2.68305738e-04],
       [ 1.76339038e-03,  1.79643765e+00, -2.11092584e+00,
         3.45964773e-04],
       [ 1.48721600e-03,  1.83987678e+00, -1.86723071e+00,
         0.00000000e+00],
       [ 1.09679988e-03,  2.10639127e+00, -1.42971128e+00,
      

In [131]:
transformed_jets[0]

array([[ 7.03576131e-01, -3.34436119e-02,  8.48179701e-01,
         1.20309630e-02],
       [ 2.16898474e-01,  8.65528729e-02,  5.98702478e-01,
         7.70915789e-03],
       [ 1.89505373e-02,  1.77311287e-01,  1.04469645e+00,
         1.53677432e-03],
       [ 1.53381211e-02,  2.47826161e-01,  9.36392854e-01,
         1.28216450e-03],
       [ 1.32579025e-02,  1.10788472e-01,  6.92954969e-01,
         7.10652757e-04],
       [ 9.37008207e-03,  6.06691248e-02,  8.84215458e-01,
         9.56582848e-04],
       [ 3.59286550e-03, -8.90246265e-01,  5.12445276e-01,
         4.98355408e-04],
       [ 3.44217527e-03, -5.27440804e-01,  7.65432685e-01,
         2.68313116e-04],
       [ 3.15475526e-03,  1.89641252e-02,  7.16633355e-01,
         2.68305738e-04],
       [ 1.76339038e-03,  3.73413085e-01,  6.42675661e-01,
         3.45964773e-04],
       [ 1.48721600e-03,  4.16852218e-01,  8.86370795e-01,
         0.00000000e+00],
       [ 1.09679988e-03,  6.83366706e-01,  1.32389022e+00,
      

In [111]:
sumjet_cartesian

array([[  1.        ,  75.13997711, -38.57886604,   2.21465659],
       [  1.        ,  15.63506297, -38.03416826,   1.06211887]])

## Generate training data

Input data is 2D, generated in the shape of a banana (plotted below) defined by two gaussians with widths 1 and 0.1. The VAE will be tasked with reconstructing the 2D location of the input points, using euclidean distance as the reconstruction error. The latent space is 2D, so can in principle easily encode everything about the input. This will be regulated by the variational latent space

In [3]:
# path to file
fn =  '/home/jcollins/projects/EMD_VAE/in_data/monoW-data.h5'

In [80]:
# Option 1: Load everything into memory
df = pandas.read_hdf(fn,stop=1000000)
print(df.shape)
print("Memory in GB:",sum(df.memory_usage(deep=True)) / (1024**3)+sum(df.memory_usage(deep=True)) / (1024**3))

(430733, 200)
Memory in GB: 1.290102794766426


In [163]:
data = df.values.reshape((-1,50,4))

HT = np.sum(data[:,:,0],axis=-1)
data[:,:,0] = data[:,:,0]/HT[:,None]
data[:,:,-1] = data[:,:,-1]/HT[:,None]

data = center_jets_ptetaphiE(data)

sig_input = np.zeros((len(data),50,5))
sig_input[:,:,:2] = data[:,:,:2]
sig_input[:,:,2] = np.cos(data[:,:,2])
sig_input[:,:,3] = np.sin(data[:,:,2])
sig_input[:,:,4] = np.log(data[:,:,3]+1e-8)

data_x = sig_input
data_y = data[:,:,:3]


train_x = data_x[:300000]
train_y = data_y[:300000]
valid_x = data_x[300000:400000]
valid_y = data_y[300000:400000]

<ipython-input-158-c1d138f24178>:36: RuntimeWarning: invalid value encountered in true_divide
  y = 0.5*np.nan_to_num(np.log((E+pz)/(E-pz)))
<ipython-input-158-c1d138f24178>:57: RuntimeWarning: invalid value encountered in true_divide
  eta = np.nan_to_num(np.arcsinh(np.sinh(y)*np.sqrt(1+np.square(m/pt))))


In [174]:
experiment_name = 'W-test'
train_output_dir = create_dir(osp.join(output_dir, experiment_name))
vae, encoder, decoder = build_and_compile_annealing_vae(optimizer=keras.optimizers.Adam(lr=0.001,clipnorm=0.1),
                                    encoder_conv_layers = [2048,2048,1028,1024],
                                    dense_size = [1028,1028,1028,512],
                                    decoder = [4026,2048,1028,512,512],
                                    numItermaxinner = 10,
                                    numIter=10,
                                    reg_init = 1.,
                                    reg_final = 0.01,
                                    stopThr=1e-3,
                                    num_inputs=5,
                                    num_particles_in=50)

batch_size=100
save_period=2

reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=1e-4, cooldown=0, min_lr=0)
modelcheckpoint = keras.callbacks.ModelCheckpoint(train_output_dir + '/model_weights_{epoch:02d}.hdf5', save_freq = save_period*5000, save_weights_only=True)
reset_metrics_inst = reset_metrics()

callbacks=[tf.keras.callbacks.CSVLogger(train_output_dir + '/log.csv', separator=",", append=True),
            reduceLR,
            modelcheckpoint,
            reset_metrics_inst]

Model: "VAE"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 50, 5)]      0                                            
__________________________________________________________________________________________________
conv1d_40 (Conv1D)              (None, 50, 2048)     12288       inputs[0][0]                     
__________________________________________________________________________________________________
re_lu_126 (ReLU)                (None, 50, 2048)     0           conv1d_40[0][0]                  
__________________________________________________________________________________________________
conv1d_41 (Conv1D)              (None, 50, 2048)     4196352     re_lu_126[0][0]                  
________________________________________________________________________________________________

In [175]:
batch_size=100
save_period=2

reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=1e-4, cooldown=0, min_lr=0)
modelcheckpoint = keras.callbacks.ModelCheckpoint(train_output_dir + '/model_weights_{epoch:02d}.hdf5', save_freq = save_period*5000, save_weights_only=True)
reset_metrics_inst = reset_metrics()

callbacks=[tf.keras.callbacks.CSVLogger(train_output_dir + '/log.csv', separator=",", append=True),
            reduceLR,
            modelcheckpoint,
            reset_metrics_inst]

vae.beta.assign(0.001)
numbatches = 5000

K.set_value(vae.optimizer.lr,1e-4)
epochs = 10


history = vae.fit(x=train_x[:numbatches*batch_size], y=train_y[:numbatches*batch_size], batch_size=batch_size,
                epochs=epochs,verbose=1,#initial_epoch=int(vae.optimizer.iterations/numbatches),
                validation_data = (valid_x[:10*batch_size],valid_y[:10*batch_size]),
                callbacks = callbacks
              )

# tf.saved_model.save(vae, train_output_dir + '/mymodel.hdf5')

Epoch 1/10
3000/3000 [==============================] - 292s 97ms/step - loss: 31545.6105 - recon_loss: 0.0312 - KL loss: 364.0737 - beta: 0.0010 - val_val_loss: 3035.8828 - val_val_recon_loss: 0.0028 - val_val_KL loss: 216.0859 - val_beta: 0.0010
Epoch 2/10
3000/3000 [==============================] - 298s 99ms/step - loss: 2562.1515 - recon_loss: 0.0024 - KL loss: 208.7358 - beta: 0.0010 - val_val_loss: 1790.0328 - val_val_recon_loss: 0.0016 - val_val_KL loss: 195.8736 - val_beta: 0.0010
Epoch 3/10
3000/3000 [==============================] - 294s 98ms/step - loss: 1717.0348 - recon_loss: 0.0015 - KL loss: 188.5682 - beta: 0.0010 - val_val_loss: 1465.5890 - val_val_recon_loss: 0.0013 - val_val_KL loss: 165.7694 - val_beta: 0.0010
Epoch 4/10
3000/3000 [==============================] - 300s 100ms/step - loss: 1336.4881 - recon_loss: 0.0012 - KL loss: 159.1612 - beta: 0.0010 - val_val_loss: 1250.8861 - val_val_recon_loss: 0.0011 - val_val_KL loss: 136.8937 - val_beta: 0.0010
Epoch 5/10

In [176]:
reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=np.sqrt(0.1), patience=4, verbose=1, mode='auto', min_delta=1e-4, cooldown=0, min_lr=1e-8)
earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_val_loss', min_delta=0., patience=10, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

modelcheckpoint = keras.callbacks.ModelCheckpoint(train_output_dir + '/model_weights_{epoch:02d}_' + str(beta) + '.hdf5', save_freq = save_period*numbatches, save_weights_only=True)
reset_metrics_inst = reset_metrics()

callbacks=[tf.keras.callbacks.CSVLogger(train_output_dir + '/log.csv', separator=",", append=True),
            reduceLR,earlystop,
            modelcheckpoint,
            reset_metrics_inst]

In [ ]:
init_epoch = 10

for beta in [0.002,0.005,0.01,0.02,0.05,0.1]:
    
    vae.beta.assign(beta)
    K.set_value(vae.optimizer.lr,1e-4)
    
    my_history = vae.fit(x=train_x[:numbatches*batch_size], y=train_y[:numbatches*batch_size], batch_size=batch_size,
                epochs=1000,verbose=1,#initial_epoch=int(vae.optimizer.iterations/numbatches),
                validation_data = (valid_x[:10*batch_size],valid_y[:10*batch_size]),
                callbacks = callbacks,initial_epoch=init_epoch
              )
    init_epoch = my_history.epoch[-1]

Epoch 11/1000
3000/3000 [==============================] - 293s 98ms/step - loss: 196.3627 - recon_loss: 5.6451e-04 - KL loss: 55.2345 - beta: 0.0020 - val_val_loss: 225.5435 - val_val_recon_loss: 6.9774e-04 - val_val_KL loss: 51.1087 - val_beta: 0.0020
Epoch 12/1000
3000/3000 [==============================] - 299s 100ms/step - loss: 208.1082 - recon_loss: 6.4294e-04 - KL loss: 47.3731 - beta: 0.0020 - val_val_loss: 202.5759 - val_val_recon_loss: 6.2764e-04 - val_val_KL loss: 45.6665 - val_beta: 0.0020
Epoch 13/1000
 376/3000 [==>...........................] - ETA: 4:22 - loss: 193.4249 - recon_loss: 5.9747e-04 - KL loss: 44.0577 - beta: 0.0020

In [27]:
my_history

NameError: name 'my_history' is not defined

In [34]:
my_history.epoch

[]